## Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import nltk
from urllib.request import urlopen # Read csv file

# nltk.download('punkt')
# nltk.download('stopwords')

## Read all files

In [ ]:
# read cv file (pdf or txt)
cv = urlopen('https://raw.githubusercontent.com/iboraham/indeed-web-scraping/master/cv/cv.txt').read()

In [ ]:
# read d.scientist csv
ds = pd.read_csv('../input/data-science-job-market-in-uk/data_scientist.csv')
ds.head()

In [ ]:
# read m.learning engineers csv
ml = pd.read_csv('../input/data-science-job-market-in-uk/ml.csv')
ml.head()

## Get desc of the jobs

In [ ]:
# trim skills and reqs from desc of jobs
def empty_count(str_series):
    counter = 0
    for ind in str_series:
        if ind == []:
            counter+=1
    return counter
ds_desc=ds.Description.str.lower()
ml_desc=ml.Description.str.lower()
ds_qualifs=ds_desc.str.findall(r'((?:qualifications|requirements|desirable|skills|required|role)+(?:\:|\?|\n)(?:.|\s)*)')
ml_qualifs=ml_desc.str.findall(r'((?:qualifications|requirements|desirable|skills|required|role)+(?:\:|\?|\n)(?:.|\s)*)')
empty_count(ml_qualifs)/len(ml_qualifs)*100, empty_count(ds_qualifs)/len(ds_qualifs)*100

In [ ]:
# concat
ml_desc_ready=ml_desc.str.cat(sep=';;;')
ds_desc_ready=ds_desc.str.cat(sep=';;;')

## Calculate similarity between cv and jobs desc's

In [ ]:
# Make a text corpus containing all words of documents. Use tokenisation and stop word removal.
import string
ds_lowers = ds_desc_ready.lower()
ml_lowers = ml_desc_ready.lower()
cv_lowers = cv.lower()
cv_lowers = str(cv_lowers)

# punctuation
translator = str.maketrans('', '', string.punctuation)
ds_clean = ds_lowers.translate(translator)
ml_clean = ml_lowers.translate(translator)
cv_clean = cv_lowers.translate(translator)

# tokenize
ds_tokens = nltk.word_tokenize(ds_clean)
ml_tokens = nltk.word_tokenize(ml_clean)
cv_tokens = nltk.word_tokenize(cv_clean)


# Use stopwords
from nltk.corpus import stopwords
ds_filtered = [w for w in ds_tokens if not w in stopwords.words('english')]
ml_filtered = [w for w in ml_tokens if not w in stopwords.words('english')]
cv_filtered = [w for w in cv_tokens if not w in stopwords.words('english')]
ds = ' '.join(ds_filtered)
ml = ' '.join(ml_filtered)
cv = ' '.join(cv_filtered)


#Stemming is optional
'''
from nltk.stem.porter import *
from collections import Counter
stemmer = PorterStemmer()
ds_stemmed = []
for item in ds_filtered:
    ds_stemmed.append(stemmer.stem(item))
ml_stemmed = []
for item in ml_filtered:
    ml_stemmed.append(stemmer.stem(item))
ds = ' '.join([el[0] for el in Counter(ds_stemmed).most_common(200)])
ml = ' '.join([el[0] for el in Counter(ml_stemmed).most_common(200)])
'''

In [ ]:
#Convert the documents into tf-idf vectors
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words='english')
docs = [ds,ml,cv]

In [ ]:
# Find the cosine-similarity between them or any new document for similarity measure.
tfidf = vect.fit_transform(docs)
similarity = tfidf*tfidf.T
similarity=similarity.toarray()

# another method
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf)

In [ ]:
docs_name = ['ds','ml','cv']
results = pd.DataFrame(data=similarity,index=docs_name,columns=docs_name)
results

As a result, I should improve my resume for these fields.